In [1]:
import pandas as pd
import numpy as np

In [2]:
def processAge(x):
    if x >= 0 and x < 2:
        return 0
    elif x >= 2 and x <= 12:
        return 1
    elif x >= 13 and x <= 59:
        return 2
    elif x >= 60:
        return 3

In [5]:
df = pd.read_csv('titanic.csv')
df = df.drop(df.columns[[10]],axis = 1)
df = df.dropna(axis=0)
df = df.reset_index(drop = True)
df_1=df.copy()
df_1['AgeCode'] = df['Age'].map(processAge)

actual = df_1['AgeCode']
update = df_1.sample(int(0.1 * len(df))).index
df_1.at[update, 'AgeCode'] = np.nan # changable

myFilter_train=df_1.AgeCode.notnull()

y_train=actual[myFilter_train]
y_test=actual[update]

x_pclass=df['Pclass']
x_pclass_1=x_pclass[myFilter_train]
x_fare=df['Fare']
x_fare_1=x_fare[myFilter_train]
X_train=pd.DataFrame({'Pclass':x_pclass_1.values, 'Fare':x_fare_1.values})

x_pclass=df['Pclass']
x_pclass_1=x_pclass[update]
x_fare=df['Fare']
x_fare_1=x_fare[update]
X_test=pd.DataFrame({'Pclass':x_pclass_1.values, 'Fare':x_fare_1.values})

In [12]:
import MICE
import iMICE

In [13]:
df_2 = df_1[['AgeCode', 'Pclass', 'Fare']].copy()

In [15]:
mice_result = MICE.MICE(verbose=False, init_fill_method="median", impute_type="pmm", n_imputations=7).complete(np.matrix(df_2))

In [18]:
df_3 = pd.DataFrame(mice_result, columns = ['AgeCode', 'Pclass', 'Fare'])
predict = df_3['AgeCode'].map(lambda x: int(x))

In [22]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix  
print(accuracy_score(actual[update], predict[update]))
print(confusion_matrix(actual[update], predict[update])) 

0.704225352112676
[[ 0  0  0  0]
 [ 1  0  4  0]
 [ 7  6 50  1]
 [ 0  0  2  0]]


# iMICE

In [27]:
imice_result = iMICE.iMICE(verbose=False, init_fill_method="median", impute_type="pmm", n_imputations=7).complete(np.matrix(df_2))

In [28]:
df_3i = pd.DataFrame(imice_result, columns = ['AgeCode', 'Pclass', 'Fare'])
predicti = df_3i['AgeCode'].map(lambda x: int(x))

In [29]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix  
print(accuracy_score(actual[update], predicti[update]))
print(confusion_matrix(actual[update], predicti[update])) 

0.9014084507042254
[[ 0  5  0]
 [ 0 64  0]
 [ 0  2  0]]


# KNN

In [30]:
X_test=X_test.iloc[:,:].values
X_train=X_train.iloc[:,:].values
y_train=np.array(y_train)
y_test=np.array(y_test)

In [31]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(3, weights='distance')
trained_model = clf.fit(X_train, y_train)
y_pred = trained_model.predict(X_test)

In [32]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.7323943661971831
[[ 0  0  0  0]
 [ 1  0  4  0]
 [ 3  6 52  3]
 [ 0  0  2  0]]


# Random Forest

In [33]:
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier

regressor = RandomForestClassifier(n_estimators=20, random_state=0)  
regressor.fit(X_train, y_train)  
y_pred = regressor.predict(X_test)

In [34]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.7183098591549296
[[ 0  0  0  0]
 [ 1  0  4  0]
 [ 1  8 51  4]
 [ 0  0  2  0]]
